In [44]:
import pandas as pd, numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import time
import csv

%matplotlib inline

In [7]:
location = "C:/Users/workspace/brad_public_workspace_on_win/SH_tongliang/data/11_24_bitmex分钟线_multiplied/bitmex分钟线1124/"
file_ada = "res_adau18z18乘10000000.csv"

In [ ]:
def chg_col(df):
    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
    return df

In [25]:
ada_df = pd.read_csv(location+file_ada, engine="python", header=None)
ada_df.index = pd.to_datetime(ada_df.iloc[:, 0])
ada_df = chg_col(ada_df)

In [109]:
def chg_col(df):
    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
    return df

def time_to_timestamp(timestr):
    """
    时间字符串转unix时间戳
    :param str: 时间字符串
    :return: unix时间戳，str类型
    """
    dt = datetime.strptime(str(timestr), '%Y-%m-%d %H:%M:%S')
    timestamp = time.mktime(dt.timetuple())
    return str(int(timestamp))

def myresample(df, period, min):
    convrted_df = df.resample(period).last()
    convrted_df['open'] = df['open'].resample(period).first()
    convrted_df['high'] = df['high'].resample(period).max()
    convrted_df['low'] = df['low'].resample(period).min()
    convrted_df['close'] = df['close'].resample(period).last()
    convrted_df['volume'] = df['volume'].resample(period).sum()
    # Keep rows with at least 5 non-NaN values
    convrted_df.dropna(thresh=5, inplace=True)
    convrted_df.index = convrted_df['time']
    convrted_df['time'] = pd.DatetimeIndex(time_translation(t, min) for t in convrted_df['time'])
    convrted_df['timestamp'] = [time_to_timestamp(i) for i in convrted_df['time']]
    return convrted_df

def time_translation(ltime, min):
    res_time = (datetime.strptime(ltime, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=min)).strftime(
        '%Y-%m-%d %H:%M:%S')
    return res_time

# Create a fucntion that transforms 1 min of data into 4-hour data:

def transform_1min_into_4hr(csv_file, location):
    df = pd.read_csv(location+csv_file, engine="python", header=None)
    df.index = pd.to_datetime(df.iloc[:, 0])
    df = chg_col(df)
    resampled_df = myresample(df, "4h", -59)
    resampled_df.index = resampled_df['time']
    del resampled_df['time']
    resampled_df.reset_index(inplace = True)
    resampled_df = resampled_df.iloc[:-1, :]
    return resampled_df

In [102]:
resampled_ada = myresample(ada_df, "4h", -59)
resampled_ada.index = resampled_ada['time']
del resampled_ada['time']
resampled_ada.reset_index(inplace=True)
resampled_ada = resampled_ada.iloc[:-1, :]

In [120]:
resampled_ada.to_csv(location+'11_24_4hr_ada.csv')

# Export other 4-hour data

In [110]:
resampled_trx = transform_1min_into_4hr("res_trxu18z18乘100000000.csv", location)

In [112]:
resampled_bch = transform_1min_into_4hr("res_bchu18z18乘1000.csv", location)

In [113]:
resampled_eos = transform_1min_into_4hr("res_eosu18z18乘100000.csv", location)

In [114]:
resampled_eth = transform_1min_into_4hr("res_ethu18z18乘10000.csv", location)

In [116]:
resampled_ltc = transform_1min_into_4hr("res_ltcu18z18乘10000.csv", location)

In [117]:
resampled_xrp = transform_1min_into_4hr("res_xrpu18z18乘10000000.csv", location)

In [118]:
resampled_btc = transform_1min_into_4hr("xbtusd_1m.csv", location)

In [121]:
# resampled_ada.to_csv(location+'11_24_4hr_ada.csv')
resampled_trx.to_csv(location+'11_24_4hr_trx.csv')
resampled_bch.to_csv(location+'11_24_4hr_bch.csv')
resampled_eos.to_csv(location+'11_24_4hr_eos.csv')
resampled_eth.to_csv(location+'11_24_4hr_eth.csv')
resampled_ltc.to_csv(location+'11_24_4hr_ltc.csv')
resampled_xrp.to_csv(location+'11_24_4hr_xrp.csv')
resampled_btc.to_csv(location+'11_24_4hr_btc.csv')